<!-- Autoheader begin -->
<hr/>
<div id="navtitle_1_1_jl" style="text-align:center; font-size:16px">I.1 Population Inversion in a Two-Level-System</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_1_2_lambda.ipynb">next notebook $\rightarrow$</a><br>
        <a href="jl_exercise_1_2_lambda.ipynb" style="font-size:13px">I.2 Population Transfer in a Three-Level-System with STIRAP</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_2_1_TLS.ipynb" style="font-size:13px">II.1 Population Inversion in a Two-Level-System using Parameter Optimization</a><br>
        <a href="jl_exercise_2_1_TLS.ipynb">$\downarrow$ next part $\downarrow$</a>
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Python/py_exercise_1_1_TLS.ipynb">👉 Python version</a></div>

---
<!-- Autoheader end -->

# Population Inversion in a Two-Level-System

The purpose of this notebook is to introduce you to the framework of Jupyter notebooks and to demonstrate how they can be used to simulate a simple quantum system - a two-level system. Jupyter notebooks allow to create and share interactive documents including live code and equations and to access them via a web browser. They can work with many programming languages running the calculations in the background - here we use Julia. The goal of this notebook is to allow you to gain familiarity with a typical workflow in a Jupyter notebook while learning about the paradigmatic example of light-matter interaction: "Rabi cycling" in a two-level system. Rabi cycling is a term describing the periodic excitation and de-excitation due to coherent interaction with a light field. Working on this example will also allow you to get to know some useful Julia packages for the numerical description and simulation of quantum systems.

## How to use this notebook

*You can evaluate all cells marked with `[n]:` by* **selecting it and hitting
SHIFT+ENTER** *or the play button in the top panel.*

Just go through the notebook and evaluate the cells one after another. You
can also change the cell to play around with the values and reevaluate it. If
you do so, make sure to evaluate all the cells that rely on the one you
changed. Have fun!

## Physical background

In this exercise we simulate the interaction of a two-level-system with a
laser pulse.

The Hamiltonian of the two-level-system is defined as

$$
\hat{H} = \hat{H}_0 + \hat{H}_1(t)
\;,
$$

where $\hat{H}_0$ is the time-independent Hamiltonian of the system and
$\hat{H}_1(t)=E(t) \, \hat{V}$ describes the interaction with the field $E(t)$.

Choosing the eigenstates of $\hat{H}_0$ as a basis, $\{|0⟩, |1⟩\}$, we
can represent the system Hamiltonian by

$$
\hat{H}_0 = -\frac{\omega}{2}
\begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
$$

and the interaction operator $\hat{V}$ by

$$
\hat{V} = -\mu_{01}
\begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}
\;.
$$

Here, $\omega>0$ is the energy splitting between the two levels and
$\mu_{01}$ is the transition matrix element.

As you will find out by numerically simulating this system below, driving the
system with a field $E(t)$ with a suitable frequency $\omega$ induces
resonant Rabi cycling, where the population perfectly cycles between the two
levels.

## Selected Julia packages

Matrices and vectors are a central part of Julia. This includes basic linear
algebra, with more advanced features available in the `LinearAlgebra`
standard library module.

In [ ]:
using LinearAlgebra

The [`DifferentialEquations.jl`](https://docs.sciml.ai/DiffEqDocs/stable/)
package (or its sub-package `OrdinaryDiffEq`) is the go-to solution for
solving any kind of differential equation.

In [ ]:
using OrdinaryDiffEq

As a more specialized tool,
[`QuantumControl.jl`](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/)
is a framework for formulating and solving quantum control problems, with the
[`QuantumPropagators.jl`](https://juliaquantumcontrol.github.io/QuantumPropagators.jl/stable/)
sub-package providing an interface to simulate time dynamics. We will use
this package here as it provides a convenient interface that is specific to
quantum dynamics.

In [ ]:
using QuantumPropagators

The package wraps `DifferentialEquation`/`OrdinaryDiffEq`, although
it also implements its own methods specifically for piecewise-constant
dynamics, which we will use in later examples.

The Julia package [`Plots.jl`](https://docs.juliaplots.org/latest/) is the
standard package for 2D data visualisation. It closely resembles the plotting
syntax from Matlab.

In [ ]:
using Plots

We'll set some defaults for `Plots`, like increasing the default line
width for better readability.

In [ ]:
Plots.default(
    linewidth               = 2.0,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

## Let's start!

We start with defining the time interval for the propagation. For numerical
calculations, we need to represent the time interval by a grid with a finite
amount of grid points.

Let the time grid start at `t_start=0` and end at `t_stop=50` with a total
amount of `Nt=10000` grid points. We can create such a grid with the built-in
`range` function, in combination with `collect` to create an explicit vector:

In [ ]:
t_start = 0
t_stop = 50
Nt = 10000
t = collect(range(t_start, t_stop; length=Nt));

## The model

Now we need to define the individual parts of the Hamiltonian. For the
simulation use the following parameters:

In [ ]:
ω = 10.0;
μ₀₁ = 1.0;

Note that Julia encourages the use of Unicode symbols. You can type these
with, e.g. `\omega<tab>` and `\mu<tab>\_01<tab>`. If you are unsure how to
type a particular unicode character, paste it into Julia's help (accessed by
a `?` at the beginning of a cell):

In [ ]:
? Ψ̃

We need the matrix for the time independent Hamiltonian $\hat{H}_0$ ...

In [ ]:
H₀ = -ω/2 * [
    1   0
    0  -1
]

... and the matrix for the interaction operator $\hat{V}$

In [ ]:
V = -μ₀₁ * [
     0  1
     1  0
]

Next, we need to define the electric field `E` on the time grid `t`.

We assume a Gaussian shaped pulse,

$$
E(t) = E_0 \, e^{-(t-t_0)^2 / (2 \tau^2)} \, \cos(\omega_l (t-t_0) + \phi)
$$

with the following parameters: $\omega_l=\omega$, $\phi=0$, $E_0=0.2$ and
$t_0=25$.

When choosing a value for the pulse duration $\tau$ one needs to be careful not to choose a duration larger than $7.5$, in order to ensure that the pulse fits completely onto our time grid.

In [ ]:
t₀ = 25.0
ωₗ = ω
E₀ = 0.2
ϕ = 0
τ = 2.5

E(t) = E₀ * cos(ωₗ * (t - t₀) + ϕ) * exp(- (t - t₀)^2 / (2τ^2));

Now we collect everything together and assemble the total Hamiltonian of our
system. This is where the `QuantumPropagtors` package becomes useful, as it
provides a function `hamiltonian` to construct a time-dependent object that
will be suitable for the `QuantumPropagators.propagate` function later on:

In [ ]:
H = hamiltonian(H₀, (V, E))

As a last step in setting up the model, we define the two states.

In [ ]:
Ψ₀ = ComplexF64[1, 0] # State |0⟩

It is critically important to ensure that `Ψ₀` is a complex vector. If we had
just written `Ψ₀ = [1, 0]`, Julia would have inferred it as an array of
integers, which is inappropriate for describing the coefficients of a quantum state.

In [ ]:
Ψ₁ = ComplexF64[0, 1] # State |1⟩

## Propagation and results

Before we can start with the propagation, we first need to define the
observables that we are interested in. For the present case, we are
interested in the population dynamics. To track the population of the two
levels, we define the projectors $\hat{P}_{i} = |i⟩⟨i|$.

In [ ]:
P₀ = Ψ₀ * Ψ₀'

Note that Julia uses `'` for the hermitian adjoint (which is usually denoted with a dagger)

In [ ]:
P₁ = Ψ₁ * Ψ₁'

With the observables all set up, we can use the `propagate` function provided by the
`QuantumPropagators` package to obtain the dynamics over time. We delegate
solving the Schrödinger equation to the `OrdinaryDiffEq` package by passing
it as `method`.

As the initial state of our simulation we choose the ground state,
$|0⟩$. With `storage=true`, we specify that we would like `propagate`
to return an array of the expectation values for the `observables` $P_0$,
$P_1$, i.e., the population in the states $|0⟩$, $|1⟩$. Without
`storage=true`, `propagate` would only return the final state.

In [ ]:
output = propagate(Ψ₀, H, t; method=OrdinaryDiffEq, observables=[P₀, P₁], storage=true)

Now let's plot the population dynamics:

In [ ]:
E_max = maximum(abs.(E.(t)))
plot(t, abs.(E.(t)) / E_max; color="lightgray", label="|E|")
plot!(t, real.(output[1,:]); color="#1f77b4", label="|0⟩")
plot!(t, real.(output[2,:]); color="#ff7f0e", linestyle=:dash, label="|1⟩")
plot!(; xlabel="Time", ylabel="Population")

Play with the pulse parameters and observe how this affects the population
dynamics. Can you find a combination of parameters that produces a complete
population inversion? What do you need to obtain a full Rabi cycle? What
happens if you change the frequency of the laser pulse?

## Next steps

Continue with [Exercise I.2](jl_exercise_1_2_lambda.ipynb) to learn about about slightly more advanced light-matter-interaction in a three-level system, or with [Exercise I.3](jl_exercise_1_3_chirp.ipynb) about the interaction of the same two-level-system with a *chirped* laser pulse. [Exercise II.1](jl_exercise_2_1_TLS.ipynb) explains how to find the proper parameters to achieve the population inversion discussed above with a gradient-free optimization. [Exercise III.1](jl_exercise_3_1_TLS.ipynb) does the same with a gradient-based approach (Krotov's method and GRAPE).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_1_1_jl)
<!-- Autofooter end -->